# Start

In [204]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [205]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
from joblib import Parallel, delayed
import os
import multiprocessing
import re

# B011

In [ ]:
# For Luis
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/B011/konbert-export-318a5f5ddb784.csv")
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/B011/konbert-export-3234d27cb7484.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
ozone_data

,index,Samplingpoint,Pollutant,Start,End,Value,Unit,AggType,Validity,Verification,ResultTime,DataCapture,FkObservationLog
0,0,BE/SPO-BETB011_00007_100,7,1992-01-01T00:00:00,1992-01-01T01:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
1,1,BE/SPO-BETB011_00007_100,7,1992-01-01T01:00:00,1992-01-01T02:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
2,2,BE/SPO-BETB011_00007_100,7,1992-01-01T02:00:00,1992-01-01T03:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
3,3,BE/SPO-BETB011_00007_100,7,1992-01-01T03:00:00,1992-01-01T04:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
4,4,BE/SPO-BETB011_00007_100,7,1992-01-01T04:00:00,1992-01-01T05:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271603,87643,BE/SPO-BETB011_00007_100,7,2022-12-31T20:00:00,2022-12-31T21:00:00,58.5,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,b07c3753-88b9-49b4-b7de-0e9348981e6c
271604,87644,BE/SPO-BETB011_00007_100,7,2022-12-31T21:00:00,2022-12-31T22:00:00,59.0,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,b07c3753-88b9-49b4-b7de-0e9348981e6c
271605,87645,BE/SPO-BETB011_00007_100,7,2022-12-31T22:00:00,2022-12-31T23:00:00,59.5,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,b07c3753-88b9-49b4-b7de-0e9348981e6c
271606,87646,BE/SPO-BETB011_00007_100,7,2022-12-31T23:00:00,2023-01-01T00:00:00,61.5,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,b07c3753-88b9-49b4-b7de-0e9348981e6c


In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data

,Start,Concentration
0,2007-01-01 00:00:00,63.0
1,2007-01-01 01:00:00,75.0
2,2007-01-01 02:00:00,74.0
3,2007-01-01 03:00:00,71.0
4,2007-01-01 04:00:00,74.0
...,...,...
113802,2019-12-31 19:00:00,2.0
113803,2019-12-31 20:00:00,3.0
113804,2019-12-31 21:00:00,3.0
113805,2019-12-31 22:00:00,2.5


In [ ]:
data['Concentration'].describe()

count    110706.000000
mean         42.252958
std          29.307746
min           2.000000
25%          19.000000
50%          41.000000
75%          60.000000
max         237.000000
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

97.275211542348


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
data['Concentration'].describe()

count    4701.000000
mean       59.492249
std        30.071590
min         2.000000
25%        40.000000
50%        58.187500
75%        75.812500
max       199.937500
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

99.13538591311682


# M705

In [ ]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/M705/konbert-export-18dd78dd1b504.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/M705/konbert-export-de8dd5be299c4.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
ozone_data

,index,Samplingpoint,Pollutant,Start,End,Value,Unit,AggType,Validity,Verification,ResultTime,DataCapture,FkObservationLog
0,0,BE/SPO-BETM705_00007_100,7,1999-01-01T00:00:00,1999-01-01T01:00:00,21.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
1,1,BE/SPO-BETM705_00007_100,7,1999-01-01T01:00:00,1999-01-01T02:00:00,20.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
2,2,BE/SPO-BETM705_00007_100,7,1999-01-01T02:00:00,1999-01-01T03:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
3,3,BE/SPO-BETM705_00007_100,7,1999-01-01T03:00:00,1999-01-01T04:00:00,26.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
4,4,BE/SPO-BETM705_00007_100,7,1999-01-01T04:00:00,1999-01-01T05:00:00,29.0,ug.m-3,hour,1,1,2016-08-17T07:51:15.530,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209299,87643,BE/SPO-BETM705_00007_100,7,2022-12-31T20:00:00,2022-12-31T21:00:00,57.500000000000000000,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,0fa1d74e-1df8-4f03-8f8c-47c13f11215c
209300,87644,BE/SPO-BETM705_00007_100,7,2022-12-31T21:00:00,2022-12-31T22:00:00,58.500000000000000000,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,0fa1d74e-1df8-4f03-8f8c-47c13f11215c
209301,87645,BE/SPO-BETM705_00007_100,7,2022-12-31T22:00:00,2022-12-31T23:00:00,59.000000000000000000,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,0fa1d74e-1df8-4f03-8f8c-47c13f11215c
209302,87646,BE/SPO-BETM705_00007_100,7,2022-12-31T23:00:00,2023-01-01T00:00:00,61.000000000000000000,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,0fa1d74e-1df8-4f03-8f8c-47c13f11215c


In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    105666.000000
mean         42.710285
std          27.337211
min           0.000000
25%          21.000000
50%          43.000000
75%          61.500000
max         210.500000
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

93.61660656856057


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
data['Concentration'].describe()

count    4548.000000
mean       58.150574
std        26.444532
min         0.625000
25%        42.125000
50%        59.562500
75%        73.875000
max       193.312500
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

96.70423134169678


# N035

In [ ]:
# For Luis
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/N035/konbert-export-2a31827ce9ca4.csv")
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/N035/konbert-export-f47afde7f8834.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
data

,Start,Concentration
0,2007-01-01 00:00:00,59.0
1,2007-01-01 01:00:00,69.0
2,2007-01-01 02:00:00,71.0
3,2007-01-01 03:00:00,68.0
4,2007-01-01 04:00:00,72.0
...,...,...
112746,2019-12-31 19:00:00,5.0
112747,2019-12-31 20:00:00,7.0
112748,2019-12-31 21:00:00,5.0
112749,2019-12-31 22:00:00,4.0


In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data['Concentration'].describe()

count    105606.000000
mean         46.769237
std          29.706065
min           0.000000
25%          26.000000
50%          45.500000
75%          64.000000
max         259.000000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

93.66302737891459


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    4532.000000
mean       62.878779
std        30.896922
min         0.000000
25%        43.687500
50%        60.562500
75%        79.250000
max       210.625000
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

96.46658152405278


# N043

In [ ]:
# For Luis
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/N043/konbert-export-8540092429b84.csv")
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/N043/konbert-export-ee042f1957ab4.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
ozone_data

,index,Samplingpoint,Pollutant,Start,End,Value,Unit,AggType,Validity,Verification,ResultTime,DataCapture,FkObservationLog
0,0,BE/SPO-BETN043_00007_100,7,1998-01-01T00:00:00,1998-01-01T01:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
1,1,BE/SPO-BETN043_00007_100,7,1998-01-01T01:00:00,1998-01-01T02:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
2,2,BE/SPO-BETN043_00007_100,7,1998-01-01T02:00:00,1998-01-01T03:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
3,3,BE/SPO-BETN043_00007_100,7,1998-01-01T03:00:00,1998-01-01T04:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
4,4,BE/SPO-BETN043_00007_100,7,1998-01-01T04:00:00,1998-01-01T05:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219043,87643,BE/SPO-BETN043_00007_100,7,2022-12-31T20:00:00,2022-12-31T21:00:00,49.0,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,3886467e-5536-49c9-8178-c97d6b2dc828
219044,87644,BE/SPO-BETN043_00007_100,7,2022-12-31T21:00:00,2022-12-31T22:00:00,49.5,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,3886467e-5536-49c9-8178-c97d6b2dc828
219045,87645,BE/SPO-BETN043_00007_100,7,2022-12-31T22:00:00,2022-12-31T23:00:00,50.5,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,3886467e-5536-49c9-8178-c97d6b2dc828
219046,87646,BE/SPO-BETN043_00007_100,7,2022-12-31T23:00:00,2023-01-01T00:00:00,54.5,ug.m-3,hour,1,1,2023-09-05T16:09:23,0.0,3886467e-5536-49c9-8178-c97d6b2dc828


In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
data

,Start,Concentration
0,2007-01-01 00:00:00,49.0
1,2007-01-01 01:00:00,64.0
2,2007-01-01 02:00:00,62.0
3,2007-01-01 03:00:00,56.0
4,2007-01-01 04:00:00,62.0
...,...,...
113850,2019-12-31 19:00:00,3.0
113851,2019-12-31 20:00:00,3.0
113852,2019-12-31 21:00:00,2.5
113853,2019-12-31 22:00:00,2.5


In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data['Concentration'].describe()

count    109532.000000
mean         31.374201
std          25.733351
min           2.000000
25%           9.000000
50%          27.500000
75%          47.000000
max         233.000000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

96.20306530235825


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    4675.000000
mean       46.252487
std        27.269544
min         2.000000
25%        26.500000
50%        43.562500
75%        61.625000
max       186.625000
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

98.54553119730186


# N045

In [ ]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/N045/earlyn045.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/N045/laten045.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data['Concentration'].describe()

count    77106.000000
mean        43.139594
std         29.827396
min          1.000000
25%         20.000000
50%         42.000000
75%         61.000000
max        226.000000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

89.41692179237405


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    3331.000000
mean       60.220110
std        30.547891
min         1.000000
25%        40.375000
50%        58.375000
75%        77.500000
max       207.937500
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

92.68224819143016


# N052

In [ ]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/N052/earlyn052.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/N052/laten052.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data['Concentration'].describe()

count    105039.000000
mean         39.842425
std          26.786848
min           0.000000
25%          19.000000
50%          39.000000
75%          57.000000
max         207.000000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

93.73878898755076


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    4522.000000
mean       55.275929
std        26.606450
min         0.000000
25%        38.203125
50%        54.906250
75%        69.875000
max       191.687500
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

96.85157421289355


# N070

In [ ]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/N070/earlyn070.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/N070/laten070.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data['Concentration'].describe()

count    108062.000000
mean         33.565587
std          27.148254
min           0.000000
25%          10.000000
50%          30.500000
75%          50.500000
max         216.500000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

95.25329008259364


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    4651.000000
mean       50.559517
std        27.107361
min         0.000000
25%        31.687500
50%        48.687500
75%        65.875000
max       199.062500
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

98.39221493547706


# R012

In [ ]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R012/earlyr012.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R012/later012.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data

,Start,Concentration
0,2007-01-01 00:00:00,62.0
1,2007-01-01 01:00:00,75.0
2,2007-01-01 02:00:00,73.0
3,2007-01-01 03:00:00,71.0
4,2007-01-01 04:00:00,76.0
...,...,...
113778,2019-12-31 19:00:00,4.5
113779,2019-12-31 20:00:00,4.0
113780,2019-12-31 21:00:00,6.0
113781,2019-12-31 22:00:00,7.0


In [ ]:
data['Concentration'].describe()

count    109822.000000
mean         45.656672
std          28.489931
min           2.000000
25%          24.500000
50%          45.000000
75%          62.500000
max         243.000000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

96.51881212483411


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    4680.000000
mean       61.727818
std        29.217854
min         2.000000
25%        43.046875
50%        60.125000
75%        77.812500
max       204.375000
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

98.71335161358363


# R222

In [208]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R222/earlyr222.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R222/r222late.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [210]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [211]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [212]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [213]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [214]:
data['Concentration'].describe()

count    105363.000000
mean         41.760471
std          29.473076
min           0.000000
25%          17.000000
50%          40.500000
75%          60.000000
max         226.500000
Name: Concentration, dtype: float64

In [215]:
s = data.shape[0]

In [216]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

93.52714038435933


In [217]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [218]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [219]:
s = data.shape[0]

In [220]:
data['Concentration'].describe()

count    4537.000000
mean       58.640126
std        29.322270
min         0.125000
25%        39.812500
50%        57.812500
75%        75.375000
max       201.500000
Name: Concentration, dtype: float64

In [221]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

96.65530464422667


# R240

In [ ]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R240/earlyr240.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R240/later240.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data['Concentration'].describe()

count    106701.000000
mean         43.078083
std          28.961206
min           0.000000
25%          21.500000
50%          40.000000
75%          59.500000
max         253.500000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

94.07351242693281


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    4603.000000
mean       60.180725
std        29.334541
min         1.000000
25%        40.750000
50%        57.062500
75%        77.000000
max       207.937500
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

97.39737621667372


# R502

In [ ]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R502/earlyr502.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R502/later502.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data['Concentration'].describe()

count    108188.000000
mean         37.270483
std          27.932438
min           0.000000
25%          15.000000
50%          34.500000
75%          53.500000
max         224.500000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

95.42491730981257


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    4660.000000
mean       53.181398
std        28.333781
min         0.000000
25%        33.875000
50%        50.125000
75%        69.375000
max       199.312500
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

98.64521591871296


# R710

In [ ]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R710/earlyr710.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R710/later710.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data['Concentration'].describe()

count    103183.000000
mean         40.850465
std          29.223797
min           0.000000
25%          17.000000
50%          40.000000
75%          60.000000
max         241.000000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

91.68969653885458


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    4478.000000
mean       58.323010
std        28.863301
min         0.000000
25%        39.812500
50%        58.125000
75%        74.625000
max       185.750000
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

95.50010663254426


# R811

In [ ]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R811/earlyr811.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R811/later811.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data['Concentration'].describe()

count    104264.000000
mean         39.902186
std          30.428253
min           0.000000
25%          14.000000
50%          38.000000
75%          59.000000
max         244.000000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

92.31559282115758


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    4493.000000
mean       57.824046
std        31.109757
min         0.000000
25%        36.250000
50%        56.125000
75%        75.000000
max       200.375000
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

95.47386315342116


# R831

In [ ]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R831/earlyr831.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R831/later831.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data['Concentration'].describe()

count    104777.000000
mean         37.121105
std          29.320348
min           0.000000
25%          11.500000
50%          34.000000
75%          56.000000
max         230.000000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

93.00696817717811


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    4518.000000
mean       55.169198
std        29.552017
min         0.000000
25%        35.250000
50%        54.375000
75%        71.687500
max       201.312500
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

96.25053259480187


# R841

In [ ]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R841/earlyr841.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/R841/later841.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
ozone_data

,index,Samplingpoint,Pollutant,Start,End,Value,Unit,AggType,Validity,Verification,ResultTime,DataCapture,FkObservationLog
0,0,BE/SPO-BETR841_00007_100,7,2003-01-01T00:00:00,2003-01-01T01:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
1,1,BE/SPO-BETR841_00007_100,7,2003-01-01T01:00:00,2003-01-01T02:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
2,2,BE/SPO-BETR841_00007_100,7,2003-01-01T02:00:00,2003-01-01T03:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
3,3,BE/SPO-BETR841_00007_100,7,2003-01-01T03:00:00,2003-01-01T04:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
4,4,BE/SPO-BETR841_00007_100,7,2003-01-01T04:00:00,2003-01-01T05:00:00,NaN,ug.m-3,hour,-1,1,2016-08-17T07:51:15.530,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121483,35059,BE/SPO-BETR841_00007_100,7,2016-12-31T20:00:00,2016-12-31T21:00:00,4.0,ug.m-3,hour,1,1,2017-09-27T08:09:16,0.0,b71ee374-cc31-4c83-903c-476e38e42ce9
121484,35060,BE/SPO-BETR841_00007_100,7,2016-12-31T21:00:00,2016-12-31T22:00:00,13.0,ug.m-3,hour,1,1,2017-09-27T08:09:16,0.0,b71ee374-cc31-4c83-903c-476e38e42ce9
121485,35061,BE/SPO-BETR841_00007_100,7,2016-12-31T22:00:00,2016-12-31T23:00:00,12.0,ug.m-3,hour,1,1,2017-09-27T08:09:16,0.0,b71ee374-cc31-4c83-903c-476e38e42ce9
121486,35062,BE/SPO-BETR841_00007_100,7,2016-12-31T23:00:00,2017-01-01T00:00:00,10.5,ug.m-3,hour,1,1,2017-09-27T08:09:16,0.0,b71ee374-cc31-4c83-903c-476e38e42ce9


In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data['Concentration'].describe()

count    78670.000000
mean        35.663341
std         27.831956
min          1.000000
25%         12.000000
50%         33.000000
75%         53.000000
max        222.000000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

91.02795519763028


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    3401.000000
mean       51.512019
std        28.456292
min         1.000000
25%        31.687500
50%        50.250000
75%        66.312500
max       193.750000
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

94.41976679622432


# WOL1

In [ ]:
# For Luis
data1 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/WOL1/earlywol1.csv")
data2 = pd.read_csv("/content/drive/My Drive/DataThesis/CSV/WOL1/latewol1.csv")
ozone_data = pd.concat([data1,data2])
ozone_data = ozone_data.reset_index()

In [ ]:
def convert_to_float(value):
    try:
        float_value = float(value)
        if float_value < 0:
            return 0
        else:
            return float_value
    except (ValueError, TypeError):
        return np.nan

ozone_data['Value'] = ozone_data['Value'].apply(convert_to_float)

In [ ]:
ozone_data.loc[(ozone_data['Validity'] == 0) | (ozone_data['Validity'] == -1), 'Value'] = np.nan

In [ ]:
columns_remove = ['Samplingpoint','Pollutant','End','Unit','AggType','Validity','Verification','ResultTime','DataCapture','FkObservationLog','index']
ozone_data = ozone_data.drop(columns=columns_remove)

ozone_data['Start'] = pd.to_datetime(ozone_data['Start'])
data = ozone_data[(ozone_data['Start'].dt.year >= 2007) & (ozone_data['Start'].dt.year <= 2019)]
data = data.reset_index()
data = data.drop(columns=['index'])
data.rename(columns={'Value': 'Concentration'}, inplace=True)

In [ ]:
duplicates = data.duplicated(subset=['Start'])

# Print or further process the duplicate values
print("Duplicate values in the column Start:", duplicates.any())

Duplicate values in the column Start: False


In [ ]:
data['Concentration'].describe()

count    107786.000000
mean         33.715557
std          24.566298
min           2.000000
25%          13.000000
50%          31.500000
75%          49.000000
max         220.500000
Name: Concentration, dtype: float64

In [ ]:
s = data.shape[0]

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

94.84947949207579


In [ ]:
# Extract year, month, day, and time into separate columns
data['Year'] = data['Start'].dt.year
data['Month'] = data['Start'].dt.month
data['Day'] = data['Start'].dt.day
data['Time'] = data['Start'].dt.time

# Extract day of the week
data['DayOfWeek'] = (data['Start'].dt.weekday + 1) % 7

# Create a new column indicating whether the day is a weekend or not
data['Weekend'] = (data['Start'].dt.dayofweek // 5).astype(int)

In [ ]:
# Calculate the 8-hour rolling mean
eight_hour_rolling_mean = data.groupby(['Year', 'Month', 'Day'])['Concentration'].rolling(window=8).mean().reset_index(drop=True)

# Add the 8-hour rolling mean as a new column in the original DataFrame
data['EightHourRollingMean'] = eight_hour_rolling_mean

# Group by 'Year', 'Month', and 'Day', and find the daily maximum of the 8-hour mean
data = data.groupby(['Year', 'Month', 'Day']).agg({'EightHourRollingMean': 'max'}).reset_index()

data.rename(columns={'EightHourRollingMean': 'Concentration', 'DayOfWeek': 'first', 'Weekend': 'first'}, inplace=True)

In [ ]:
s = data.shape[0]

In [ ]:
data['Concentration'].describe()

count    4605.000000
mean       46.919381
std        25.572938
min         2.000000
25%        28.875000
50%        45.250000
75%        61.500000
max       179.375000
Name: Concentration, dtype: float64

In [ ]:
summary_stats = data['Concentration'].describe()

# Get the count of non-null values
count_non_null = summary_stats['count']

print((count_non_null/s)*100)

97.2544878563886
